In [2]:
import sys
sys.path.append('..')
from d2l import torch as d2l
import torch
from torch import nn 

batch_size = 64
train_iter, test_iter, vocab = d2l.load_data_imdb(batch_size)

KeyboardInterrupt: 

In [5]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, **kwargs):
        super().__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, bidirectional=True)
        self.decoder = nn.Linear(4*num_hiddens, 2)
    
    def forward(self, inputs):
        embeddings = self.embedding(inputs.T)
        self.encoder.flattern_parameters()
        outputs, _ = self.encoder(embeddings)
        encoding = torch.cat((outputs[0], outputs[-1]), dim=1)
        outs = self.decoder(encoding)
        return outs

In [6]:
embed_size, num_hiddens, num_layers, devices=100, 100, 2, d2l.try_all_gpus()
net = BiRNN(len(vocab), embed_size, num_hiddens, num_layers)

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
    if type(m) == nn.LSTM:
        for param in m._flat_weights_names:
            if "weight" in param:
                nn.init.xavier_uniform_(m._parameters[param])
net.apply(init_weights)

BiRNN(
  (embedding): Embedding(49346, 100)
  (encoder): LSTM(100, 100, num_layers=2, bidirectional=True)
  (decoder): Linear(in_features=400, out_features=2, bias=True)
)

In [10]:
glove_embedding = d2l.TokenEmbedding('glove.6b.100d')

UnicodeDecodeError: 'gbk' codec can't decode byte 0x93 in position 5456: illegal multibyte sequence

In [ ]:
embeds = glove_embedding[vocab.idx_to_token]
embed.shape

In [ ]:
net.embedding.weight.data.copy_(embeds)
net.embedding.weight.requires_grad = False

In [ ]:
lr, num_epochs = 0.01, 5
trainer = torch.optim.Adam(net.parameters(), lr=lr)
loss = nn.CrossEntropyLoss(reduction="none")
d2l.train_ch14(net, train_iter, test_iter, loss, train, num_epochs, devices)

In [ ]:
def predict_sentiment(net, vocab, sentence):
    sentence = torch.tensor(vocabp[sentence.split()], device=d2l.try_gpu())
    label = torch.argmax(net(sentence.reshape(1, -1)), dim=1)
    return 'positive' if label==1 else 'negative'

In [ ]:
predict_sentiment(net, vocab, 'this movie is so great')

In [ ]:
predict_sentiment(net, vocab, 'this movie is not great')

In [ ]:
predict_sentiment(net, vocab, 'this movie is so bad')

In [ ]:
predict_sentiment(net, vocab, 'this movie is not bad ')